In [1]:
import openai
import requests
import pandas as pd

# OPENAI API KEY
openai.api_key = 'YOUR-API-KEY'

# TELEGRAM URL AND TELEGRAM CHAT ID
telegram_api_token = 'YOUR-API-TOKEN'
telegram_chat_id = 'YOUR-CHAT-ID'

#READING USER INPUT FROM TELEGRAM AND PASSING IT TO SEND_MSG FUNCTION
def read_msg(offset):
    parameters = {
        "offset": offset
    }

    resp = requests.get(f"https://api.telegram.org/bot{telegram_api_token}/getUpdates", params=parameters)
    data = resp.json()

    for result in data["result"]:
        if "message" in result and "text" in result["message"]:
            text = result["message"]["text"]
            print(f"User Question: {text}")

            message_id = result["message"]["message_id"]
            send_msg(text, message_id)

    if data["result"]:
        return data["result"][-1]["update_id"] + 1

#SENDING RESPONSE FROM CHATGPT TO THE TELEGRAM
def send_msg(text, reply_to_message_id):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": text}]
    )
    bot_reply = completion.choices[0]['message']['content']

    parameters = {
        "chat_id": telegram_chat_id,
        "text": bot_reply,
        "reply_to_message_id": reply_to_message_id
    }

    resp = requests.get(f"https://api.telegram.org/bot{telegram_api_token}/sendMessage", params=parameters)
    data = resp.json()

    if "result" in data and "text" in data["result"]:
        message_text = data["result"]["text"]
        print(f"Telegram Response: {message_text}")

offset = 0

while True:
    offset = read_msg(offset)


ModuleNotFoundError: No module named 'openai'